### Read training, dev and unlabeled test data

The following provides a starting code (Python 3) of how to read the labeled training and dev cipher text, and unlabeled test cipher text, into lists.

In [146]:
train, dev, test = [], [], []

In [147]:
for x in open('./train_enc.tsv', encoding='utf-8'):
    x = x.rstrip('\n\r').split('\t')
    # x[0] will be the label (0 or 1), and x[1] will be the ciphertext sentence.
    x[0] = int(x[0]) 
    train.append(x)
print (len(train))
print (train[:3])

16220
[[0, 'lkêcê yoúc cêêö y#êjl lw mówám Újám j Úêê# ütlk Úol lkêú z#ê ctöé8ú ówl xoóóú éê#xw#öê#c .'], [0, '6êcétlê jolêot8 zc éê#xw#öjóáê , tl zc j #jlkê# 8tcl8êcc jöÚ8ê 6wüó lkê öt668ê wx lkê #wj6 , ükê#ê lkê lkêöjltá t#wótêc j#ê lww wÚ2twoc jó6 lkê cê+oj8 éw8tltác lww cöoy .'], [0, 'tx lktc kw8t6jú öw2tê tc coééwcê6 lw Úê j ytxl , cwöêÚw6ú oóü#jééê6 tl êj#8ú , lwwm wol j88 lkê yww6 cloxx , jó6 8êxl Úêktó6 lkê á#jé ( 8tlê#j88ú ) .']]


In [148]:
for x in open('./dev_enc.tsv', encoding='utf-8'):
    x = x.rstrip('\n\r').split('\t')
    # x[0] will be the label (0 or 1), and x[1] will be the ciphertext sentence.
    x[0] = int(x[0]) 
    dev.append(x)
print (len(dev))
print (dev[:3])

2027
[[1, 'ów8jó Ú#j2ê8ú l#êj6c ükê#ê xêü jöê#tájó xt8öc 6j#ê lw 6ê82ê 77 tólw lkê üw#86 wx jöÚt2j8êóáê jó6 jöÚtyotlú <<<'], [0, 'ê2êó öo#ékú zc ê+éê#l áwötá ltötóy jó6 xjöê6 ákj#tcöj áj ózl #êcáoê lktc êxxw#l .'], [1, 'üt88 jcco#ê68ú #jóm jc wóê wx lkê á8ê2ê#êcl , öwcl 6êáêélt2ê8ú jöoctóy áwöê6têc wx lkê úêj# .']]


#### Different from 'train' and 'dev' that are both list of tuples, 'test' will be just a list.

In [149]:
for x in open('./test_enc_unlabeled.tsv', encoding='utf-8'):
    x = x.rstrip('\n\r')
    test.append(x)
print (len(test))
print (test[:3])

2028
['j 6t6jáltá jó6 6o88 6wáoöêólj#ú y8w#txútóy cwxlüj#ê jój#ákú .', 'ówlktóy cltámc , #êj88ú , ê+áêél j 8tóyê#tóy á#êêétóêcc wóê xêê8c x#wö Úêtóy 6#jyyê6 lk#woyk j cj6 , cw#6t6 oót2ê#cê wx yoóc , 6#oyc , j2j#táê jó6 6jöjyê6 6#êjöc .', 'öo#ékú jó6 üt8cwó jáloj88ú öjmê j é#êllú yww6 lêjö <<< Úol lkê é#wvêál co##woó6tóy lkêö tc 6tcl#êcctóy8ú #wlê .']


#### You can split every sentence into lists of words by white spaces.

In [150]:
train_split = [[x[0], x[1].split(' ')] for x in train]
dev_split = [[x[0], x[1].split(' ')] for x in dev]
test_split = [[x.split(' ')] for x in test]

### Main Code Body

You may choose to experiment with different methods using your program. However, you need to embed the training and inference processes at here. We will use your prediction on the unlabeled test data to grade, while checking this part to understand how your method has produced the predictions.

Importing the Required Libraries.

In [151]:
import numpy as np
import pandas as pd 
from sklearn.model_selection import train_test_split
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential, load_model
from keras.layers import Dropout
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D
from keras.callbacks import EarlyStopping, ModelCheckpoint

In [152]:
def get_train_val_test_data(train_split,dev_split,test_split):

  train_data = pd.DataFrame(train_split, columns=['class_label', 'cipher_sentence'])
  # converting the token representation to sentence representation.
  train_data['cipher_sentence'] = [" ".join(t) for t in train_data['cipher_sentence'].values]

  validation_data = pd.DataFrame(dev_split, columns=['class_label', 'cipher_sentence'])
  # converting the token representation to sentence representation.
  validation_data['cipher_sentence'] = [" ".join(t) for t in validation_data['cipher_sentence'].values]

  test_data = pd.DataFrame(test_split, columns=['cipher_sentence'])
  # converting the token representation to sentence representation.
  test_data['cipher_sentence'] = [" ".join(t) for t in test_data['cipher_sentence'].values]

  return train_data,validation_data,test_data

train_data,validation_data,test_data = get_train_val_test_data(train_split,dev_split,test_split)

In [153]:
train_data.head(4)

,class_label,cipher_sentence
0,0,lkêcê yoúc cêêö y#êjl lw mówám Újám j Úêê# ütl...
1,0,"6êcétlê jolêot8 zc éê#xw#öjóáê , tl zc j #jlkê..."
2,0,tx lktc kw8t6jú öw2tê tc coééwcê6 lw Úê j ytxl...
3,1,vocl ükêó úwo lktóm lkjl ê2ê#ú éwcctÚ8ê jóy8ê ...


In [154]:
validation_data.head(4)

,class_label,cipher_sentence
0,1,ów8jó Ú#j2ê8ú l#êj6c ükê#ê xêü jöê#tájó xt8öc ...
1,0,ê2êó öo#ékú zc ê+éê#l áwötá ltötóy jó6 xjöê6 á...
2,1,"üt88 jcco#ê68ú #jóm jc wóê wx lkê á8ê2ê#êcl , ..."
3,1,"tl kjc j ájxxêtójlê6 , c8wééú Ú#t88tjóáê , céj..."


In [155]:
test_data.head(4)

,cipher_sentence
0,j 6t6jáltá jó6 6o88 6wáoöêólj#ú y8w#txútóy cwx...
1,"ówlktóy cltámc , #êj88ú , ê+áêél j 8tóyê#tóy á..."
2,öo#ékú jó6 üt8cwó jáloj88ú öjmê j é#êllú yww6 ...
3,lkê xt8ö üjc é#w6oáê6 Úú vê##ú Ú#oámkêtöê# jó6...


In [156]:
print('Shape of Train data ',train_data.shape)
print('Shape of Validation data ',validation_data.shape)
print('Shape of Test data ',test_data.shape)

Shape of Train data  (16220, 2)
Shape of Validation data  (2027, 2)
Shape of Test data  (2028, 1)


In [157]:
def get_hyperparameters():
  return 70000,350,100,0.25,'adam',5,32,0.03

max_words, max_length_of_seq,dimensions_of_emb,dropout,optimizer,num_epochs,batch,validation_split_up = get_hyperparameters()

In [159]:
def process_data(train_data,validation_data):
  tokenzing_text = Tokenizer(num_words=max_words,lower=True)
  tokenzing_text.fit_on_texts(train_data['cipher_sentence'].values)
  word_index = tokenzing_text.word_index
  t_data_x = tokenzing_text.texts_to_sequences(train_data['cipher_sentence'].values)
  t_data_x = pad_sequences(t_data_x, maxlen=max_length_of_seq)
  
  v_data_x = tokenzing_text.texts_to_sequences(validation_data['cipher_sentence'].values)
  v_data_x = pad_sequences(v_data_x, maxlen=max_length_of_seq)
  return t_data_x, v_data_x, tokenzing_text

t_data_x, v_data_x, tokenzing_text = process_data(train_data,validation_data)

train_data_x, val_data_x, train_data_y, val_data_y = t_data_x,v_data_x,pd.get_dummies(train_data['class_label']).values,pd.get_dummies(validation_data['class_label']).values

In [160]:
train_data_y

array([[1, 0],
       [1, 0],
       [1, 0],
       ...,
       [0, 1],
       [1, 0],
       [1, 0]], dtype=uint8)

In [161]:
def lstm(train_data_x,train_data_y):
  max_words, max_length_of_seq,dimensions_of_emb,dropout,optimizer,num_epochs,batch,validation_split_up = get_hyperparameters()

  lstm_model = Sequential()
  lstm_model.add(Embedding(max_words, dimensions_of_emb, input_length = train_data_x.shape[1]))
  lstm_model.add(LSTM(50, activation = 'tanh', recurrent_activation = 'hard_sigmoid', dropout = dropout, recurrent_dropout = dropout, return_sequences = True))
  lstm_model.add(LSTM(20, activation = 'tanh', recurrent_activation = 'hard_sigmoid', dropout = 0.35, recurrent_dropout = 0.35))
  lstm_model.add(Dense(2, activation = 'softmax'))
  lstm_model.compile(loss = 'categorical_crossentropy', optimizer = optimizer, metrics=['accuracy'])
  
  epochs = num_epochs
  batch_size = batch
  full_data = lstm_model.fit(train_data_x, train_data_y, epochs=epochs, batch_size = batch_size, validation_data=(val_data_x,val_data_y),callbacks=[EarlyStopping(monitor='val_accuracy', patience=3, min_delta=0.0001)])
  return lstm_model

lstm_model = lstm(train_data_x,train_data_y)

Epoch 1/5
507/507 [==============================] - 478s 929ms/step - loss: 0.5053 - accuracy: 0.7472 - val_loss: 0.3672 - val_accuracy: 0.8481
Epoch 2/5
507/507 [==============================] - 465s 918ms/step - loss: 0.2541 - accuracy: 0.9019 - val_loss: 0.3349 - val_accuracy: 0.8653
Epoch 3/5
507/507 [==============================] - 461s 909ms/step - loss: 0.1558 - accuracy: 0.9448 - val_loss: 0.3838 - val_accuracy: 0.8791
Epoch 4/5
507/507 [==============================] - 457s 902ms/step - loss: 0.1116 - accuracy: 0.9597 - val_loss: 0.3476 - val_accuracy: 0.8811
Epoch 5/5
507/507 [==============================] - 459s 906ms/step - loss: 0.0822 - accuracy: 0.9682 - val_loss: 0.4763 - val_accuracy: 0.8885


In [162]:
def get_dev_set_accurcy(accurate_lstm_model,tokenzing_text):

  file=open('./dev_enc.tsv','r')
  lines = file.readlines()
  ground_labels,ground_sentences=[],[]
  for line in lines:
    ground_labels.append(line.split()[0])
    ground_sentences.append(" ".join(line.split()[1:]))

  results = []
  for sentence in ground_sentences:
    val_data = [sentence]
    temp_seq = tokenzing_text.texts_to_sequences(val_data)
    data_padded = pad_sequences(temp_seq, maxlen=max_length_of_seq)
    predicted = accurate_lstm_model.predict(data_padded)
    class_labels = ['0', '1']
    results.append(class_labels[np.argmax(predicted)])
  
  total = len(ground_labels)
  correct =0
  for i in range(len(ground_labels)):
    if results[i]==ground_labels[i]:
      correct+=1
  print('Total correct :',correct)
  print('Accuracy :',correct/total)

  return results

results = get_dev_set_accurcy(lstm_model,tokenzing_text)

Total correct : 1801
Accuracy : 0.8885051800690675


In [163]:
num_of_ones,num_of_zeros = 0,0
for i in results:
  if i=='1':
    num_of_ones+=1
  else:
    num_of_zeros+=1
print('Dev set Split up')
print('1 :',num_of_ones)
print('0 :',num_of_zeros)

Dev set Split up
1 : 1079
0 : 948


In [164]:
def get_test_set_accurcy(accurate_lstm_model,tokenzing_text):

  file=open('./test_enc_unlabeled.tsv','r')
  lines = file.readlines()
  ground_sentences=[]
  for line in lines:
    ground_sentences.append(" ".join(line.split()[1:]))

  results = []
  for sentence in ground_sentences:
    test_data = [sentence]
    temp_seq = tokenzing_text.texts_to_sequences(test_data)
    data_padded = pad_sequences(temp_seq, maxlen=max_length_of_seq)
    predicted = accurate_lstm_model.predict(data_padded)
    class_labels = ['0', '1']
    results.append(class_labels[np.argmax(predicted)])
  
  return results

results = get_test_set_accurcy(lstm_model,tokenzing_text)

In [165]:
num_of_ones,num_of_zeros = 0,0
for i in results:
  if i=='1':
    num_of_ones+=1
  else:
    num_of_zeros+=1
print('Test set Split up')
print('1 :',num_of_ones)
print('0 :',num_of_zeros)

Test set Split up
1 : 1016
0 : 1012


### Output Prediction Result File

You will need to submit a prediction result file. It should have 2028 lines, every line should be either 0 or 1, which is your model's prediction on the respective test set instance.

In [166]:
# suppose you had your model's predictions on the 2028 test cases read from test_enc_unlabeled.tsv, and 
#those results are in the list called 'results'
assert (len(results) == 2028)

In [167]:
# make sure the results are not float numbers, but intergers 0 and 1
results = [int(x) for x in results]

In [168]:
# write your prediction results to 'upload_predictions.txt' and upload that later
with open('upload_predictions.txt', 'w', encoding = 'utf-8') as fp:
    for x in results:
        fp.write(str(x) + '\n')